# Prototyping Controlled Mathematical Languages in Jupyter Notebooks

**Presentation for ICMS 2020**


## Initial Remarks

This notebooks contains the core part of the presentation and shows off the various features of the Jupyter kernel with a minimal example.
The recorded presentation also briefly shows other notebooks for more complex examples and contains some (LaTeX) slides for the introduction and conclusion.
Below is an alternative introduction.

## Alternative Introduction

A *controlled mathematical language* is a formal language (with fixed semantics) that imitates natural language.
They could be used to decrease the barrier of entry to users of mathematical software.
We believe that this is rarely done in practice because designing such languages is very challenging.
We also believe that the Grammatical Logical Framework (GLF) can be used to prototype such languages and present a new Jupyter interface for GLF.

#### What is GLF?
GLF combines two existing frameworks:
* GF (the Grammatical Framework), which is a great tool for developing natural language grammars,
* and MMT, which can be used for logic development.

Together, they allow us to specify a language-to-logic pipeline by describing

* A grammar
* A target logic
* A semantics construction (which maps abstract syntax trees to logical expressions)

#### What does the kernel do?

It starts GF and MMT in the background.
Pattern matching is used to distinguish different types of contents:
* GF grammar modules are written to files and imported into the GF shell
* MMT content is also written to files and gets built
* Commands are either handed to the GF shell or are handled directly by the kernel

### Part 1: Grammar

#### Abstract Syntax

In [21]:
abstract MiniGrammar = {
    cat
        Question; Term;
    fun
        askTerm  : Term -> Question;
        intTerm  : Int  -> Term;
        multiply : Term -> Term -> Term;
}

Defined MiniGrammar

In [22]:
generate_random -cat=Question

askTerm (intTerm 999)

#### Concrete Syntax

Describes mapping to strings. Enter `MiniGrammarEng` and press <kbd>Tab</kbd> for stub generation.

In [23]:
MiniGrammarEng

Input is neither valid GF or MMT content nor a valid shell command!

In [24]:
concrete MiniGrammarEng of MiniGrammar = {
    lincat
        Question = Str;
        Term = Str;
    lin
        askTerm t = "what is" ++ t;
        intTerm i = i.s;
        multiply a b = a ++ "*" ++ b;
}

In [25]:
parse -cat=Question "what is 2 * 3"

askTerm (multiply (intTerm 2) (intTerm 3))

#### Concatenating Commands

In [26]:
generate_random -cat=Question -depth=3

askTerm (intTerm 999)

In [27]:
linearize askTerm (multiply (intTerm 999) (intTerm 999))

what is 999 * 999

In [28]:
generate_random -cat=Question -depth=3 -number=5 | linearize

what is 999

what is 999

what is 999 * 999

what is 999 * 999

what is 999 * 999

#### Visualization Trees

In [29]:
vt askTerm (multiply (intTerm 2) (intTerm 3))

graph {

n0[label = "askTerm : Question", style = "solid", shape = "plaintext"] ;

n0_0[label = "multiply : Term", style = "solid", shape = "plaintext"] ;

n0 -- n0_0 [style = "solid"];

n0_0_0[label = "intTerm : Term", style = "solid", shape = "plaintext"] ;

...

n1_0_0[label = "intTerm : Term", style = "solid", shape = "plaintext"] ;

n0_0 -- n1_0_0 [style = "solid"];

n0_1_0_0[label = "3", style = "solid", shape = "plaintext"] ;

n1_0_0 -- n0_1_0_0 [style = "solid"];

}

In [30]:
parse -cat=Question "what is 2 * 3" | vt

graph {
n0[label = "askTerm : Question", style = "solid", shape = "plaintext"] ;
n0_0[label = "multiply : Term", style = "solid", shape = "plaintext"] ;
n0 -- n0_0 [style = "solid"];
n0_0_0[label = "intTerm : Term", style = "solid", shape = "plaintext"] ;
n0_0 -- n0_0_0 [style = "solid"];
n0_0_0_0[label = "2", style = "solid", shape = "plaintext"] ;
n0_0_0 -- n0_0_0_0 [style = "solid"];
n1_0_0[label = "intTerm : Term", style = "solid", shape = "plaintext"] ;
n0_0 -- n1_0_0 [style = "solid"];
n0_1_0_0[label = "3", style = "solid", shape = "plaintext"] ;
n1_0_0 -- n0_1_0_0 [style = "solid"];
}

In [31]:
parse -cat=Question "what is 2 * 3" | vt | show

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01k\x00\x00\x01[\x08\x06\x00\x00\x00\x13<\xaf\xd5\x…

#### Handling Ambiguities

Two ways of putting parentheses:
* (2 * 3) * 4
* 2 * (3 * 4)

In [32]:
parse -cat=Question "what is 2 * 3 * 4"

askTerm (multiply (intTerm 2) (multiply (intTerm 3) (intTerm 4)))

askTerm (multiply (multiply (intTerm 2) (intTerm 3)) (intTerm 4))

In [33]:
parse -cat=Question "what is 2 * 3 * 4" | vp -showfun | show

2 graphs generated

Dropdown(description='Tree of:', layout=Layout(width='max-content'), options=('askTerm (multiply (intTerm 2) (…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\x1c\x00\x00\x02\x1b\x08\x06\x00\x00\x00au\x98\x0…

### Part 2: Target Logic

<!-- Target logic and domain theory are entered as MMT theories. -->

Unicode characters can be entered via <kbd>Tab</kbd> completion (e.g. `\forall`<kbd>Tab</kbd>).

In [34]:
theory Target : ur:?LF =
    include ☞http://cds.omdoc.org/urtheories?NatSymbols ❙
    multiplication : NAT ⟶ NAT ⟶ NAT ❘ # 1 ⋅ 2 ❙
❚

Created theory Target

### Part 3: Semantics Construction

Mapping ASTs to logical expressions. Enter `MiniGrammarSemantics` and press <kbd>Tab</kbd> for stub generation.

In [35]:
MiniGrammarSemantics

Input is neither valid GF or MMT content nor a valid shell command!

In [36]:
view MiniGrammarSemantics : http://mathhub.info/comma/jupyter/MiniGrammar.gf?MiniGrammar -> ?Target =
    NAT = NAT ❙
    Question = NAT ❙
    Term = NAT ❙

    askTerm = [x] x ❙
    intTerm = [x] x ❙
    multiply = [a,b] a ⋅ b ❙
❚

Created view MiniGrammarSemantics

### Part 4: Putting it all together

In [37]:
parse -cat=Question "what is 2 * 3"

askTerm (multiply (intTerm 2) (intTerm 3))

In [38]:
parse -cat=Question "what is 2 * 3" | construct -e

(multiplication 2 3)

In [39]:
parse -cat=Question "what is 2 * 3" | construct

2⋅3

In [40]:
parse -cat=Question "what is 2 * 3 * 4" | construct

2⋅(3⋅4)

(2⋅3)⋅4